In [1]:
CPU_CORES = 51
addprocs(CPU_CORES - 1);
@everywhere using DistributedArrays, JuMP, Distributions, Ipopt
using StatsBase

} where T) in module Base at deprecated.jl:56 overwritten in module StatsBase at /home/cqgz21/.julia/v0.6/StatsBase/src/hist.jl:535.


In [2]:
print(nprocs(),"\n")
print(nworkers(),"\n")
print(workers(),"\n")

51
50
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [3]:
@everywhere landscape1= [0. 0 0 2 2 0 0 0 2 0 0 2;
              2 0 1 2 5 5 0 0 2 0 0 0;
              3 1 2 2 1 1 2 0 0 3 0 0;
              2 1 1 2 1 2 2 0 3 4 4 0
              2 2 1 1 1 1 1 0 3 4 4 3;
              2 2 1 1 1 1 1 1 3 3 3 0;
              0 2 0 1 5 5 5 0 1 1 0 3;
              3 0 0 1 0 0 2 0 0 3 1 3;
              3 4 0 1 1 3 2 2 2 3 3 3;
              4 3 2 0 0 3 2 2 2 3 2 2] + 1
@everywhere Ncell = 30

In [4]:
@everywhere function create_pixel_mat(landscape, ordered_vec)
    retmat = deepcopy(landscape)
    for i = 1:length(ordered_vec)
        retmat[abs.(retmat - i) .< 0.000000001] = ordered_vec[i]
    end
    
    if size(retmat) == (10,12)
        retmat_zone = hcat(reshape(retmat[1:5 , 1:6],Ncell),
            reshape(retmat[1:5 , 7:12],Ncell),
            reshape(retmat[6:10 , 1:6],Ncell),
            reshape(retmat[6:10 , 7:12],Ncell))
    end
    retmat_zone
end


In [5]:
# Using the prior and visibility info given in the DCA paper
# These have @everywhere because they are fixed. Better declare them in each process rather than
# copying from the parent process
@everywhere s1 = create_pixel_mat(landscape1, [0.4,0.9,0.3,0.2,0.1,0.8])
@everywhere s2 = create_pixel_mat(landscape1, [0.5,0.1,0.1,0.7,0.6,0.9])
@everywhere s3 = create_pixel_mat(landscape1, [0.6,0.1,0.4,0.8,0.7,0.1])
@everywhere s4 = create_pixel_mat(landscape1, [0.8,0.1,0.6,0.2,0.1,0.7])
@everywhere s5 = create_pixel_mat(landscape1, [0.5,0.3,0.5,0.4,0.3,0.6])
@everywhere s6 = create_pixel_mat(landscape1, [0.1,0.5,0.2,0.6,0.5,0.2])




@everywhere Z = 4 # number of zones
@everywhere S = 6 # number of sensors
W = reshape(hcat(s1,s2,s3,s4,s5,s6),Ncell,Z,S) # visibility tensor

# Decrease censors' visibility
W = W/6



@everywhere θ = 0.3
@everywhere N = 30 # Threshold at every worker needs it
@everywhere Threshold = convert(Int64,floor(N * θ)) # for accepting P computed from each sample
@everywhere epsilon = 1e-5


# No @everywhere because these are only used by the parent process
maxiter = 15 # number of iterations for 



# No @everywhere because these are changing, pass into functions as parameters

Φ = 5
# prior_ordered_vec = [0.0085,0.001,0.0115,0.013,0.014,0] # prior of target, order given in the paper
# prior = create_pixel_mat(landscape1, prior_ordered_vec)
# print(sum(prior))





5

In [6]:



####################################
## CE Algorithm parallel N

@everywhere function solve_alloc(u, Φ,prior,W)

    m_ϕ = Model(solver = IpoptSolver(print_level=0))

    @variable(m_ϕ,ϕvar[1:Ncell,1:Z,1:S]) # resource allocation for each senor in each zone in each cell
    @variable(m_ϕ,exponential[1:Ncell,1:Z])
    @variable(m_ϕ, aux[1:Ncell,1:Z])
    @objective(m_ϕ, Min, sum(prior .* exponential))
    @NLconstraint(m_ϕ, [i=1:Ncell,j=1:Z], exp(aux[i,j]) <= exponential[i,j])
    @constraint(m_ϕ,aux .== squeeze(-sum(W .* ϕvar .* reshape(u,1,Z,S),3),3))
    @constraint(m_ϕ, exponential .>= 0)
    @constraint(m_ϕ,ϕvar .>= 0)
    @constraint(m_ϕ,[i=1:Z,j=1:S],sum(ϕvar[:,i,j]) <= Φ)

    solve(m_ϕ)
    getobjectivevalue(m_ϕ), transpose(u)
end
    
    

function CE_parallel(Φ,prior,W)
    
    P = ones(S,Z)/Z
    ind = true
    P_holder = zeros(S,Z)
    for iter=1:maxiter

        cumsumP = cumsum(P,2)
        
        vec_tile = rand(Uniform(0,1), S * N,1)
        cumsumP_tile = repmat(cumsumP,N)
        which_zone = floor.(findmax(cumsumP_tile .> vec_tile,2)[2] / (S*N) - epsilon) + 1
        u = abs.(repmat(transpose(1:Z),S*N,1) .- which_zone) .< epsilon
        u = transpose(u)
        
        arr_obj = []
        arr_assign = zeros(S,Z,N)
#         tic()
        retobj = @DArray [solve_alloc(u[:,(i-1)*S+1:i*S],Φ,prior,W) for i = 1:N]
#         toc()
        for i = 1:N
            push!(arr_obj, retobj[i][1])
            arr_assign[:,:,i] = retobj[i][2]
        end
 
        best_ind = sortperm(arr_obj)[1:Threshold]
        P = squeeze(sum(arr_assign[:,:,best_ind],3)/Threshold,3)
        
        if sum(abs.(P - 1) .<=0.000001) == S
#             print(iter,'\n')
            break
        end
    end
    
    u = transpose(P)
    m_ϕ = Model(solver = IpoptSolver(print_level=0))

    @variable(m_ϕ,ϕvar[1:Ncell,1:Z,1:S])
    @variable(m_ϕ,exponential[1:Ncell,1:Z])
    @variable(m_ϕ, aux[1:Ncell,1:Z])
    @objective(m_ϕ, Min, sum(prior .* exponential))
    @NLconstraint(m_ϕ, [i=1:Ncell,j=1:Z], exp(aux[i,j]) <= exponential[i,j])
    @constraint(m_ϕ,aux .== squeeze(-sum(W .* ϕvar .* reshape(u,1,Z,S),3),3))
    @constraint(m_ϕ, exponential .>= 0)
    @constraint(m_ϕ,ϕvar .>= 0)
    @constraint(m_ϕ,[i=1:Z,j=1:S],sum(ϕvar[:,i,j]) <= Φ)

    solve(m_ϕ)
    
    getobjectivevalue(m_ϕ), P, exp.(getvalue(aux)), getvalue(ϕvar)
    
    
    
end


CE_parallel (generic function with 1 method)

In [17]:
function map2repr(x,y)
    xoffset = [0,6,0,6]
    yoffset = [0,0,5,5]
    which_zone = 1 + (x > centerx)  + (y > centery) * 2
    xoff = x - xoffset[which_zone]
    yoff = y - yoffset[which_zone]
    which_cell = (xoff-1) * 5 + yoff
    return which_cell, which_zone
end


# Make prior at the centre of the landscape
# raidus 1, 50% 
# radius 2, 50%



prior = zeros(30,4)
radius = 2
centerx = 6.5
centery = 5.5
maxx = convert(Int64,centerx + (radius - 0.5))
minx = convert(Int64,centerx - (radius - 0.5))
maxy = convert(Int64,centery + (radius - 0.5))
miny = convert(Int64,centery - (radius - 0.5))

for x=minx:maxx
    for y=miny:maxy
        which_cell, which_zone = map2repr(x,y)
        prior[which_cell,which_zone] = 0.5 / 12
    end
end


radius = 1
centerx = 6.5
centery = 5.5
maxx = convert(Int64,centerx + (radius - 0.5))
minx = convert(Int64,centerx - (radius - 0.5))
maxy = convert(Int64,centery + (radius - 0.5))
miny = convert(Int64,centery - (radius - 0.5))

for x=minx:maxx
    for y=miny:maxy
        which_cell, which_zone = map2repr(x,y)
        prior[which_cell,which_zone] = 0.5 / 4
    end
end

In [18]:
# Test run. good practice to just run it when first open the notebook (initialize the optimizer for each process)

tic()
Φ = 5
maxiter = 20
@everywhere N = 30 # Threshold at every worker needs it
@everywhere Threshold = convert(Int64,floor(N * θ)) # for accepting P computed from each sample
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,prior,W)
toc()
print(obj,'\n')

print(assignment,'\n')

	From worker 28:	
	From worker 28:	******************************************************************************
	From worker 28:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 28:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 28:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 28:	******************************************************************************
	From worker 28:	
	From worker 23:	
	From worker 23:	******************************************************************************
	From worker 23:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 23:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 23:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 23:	******************************************************************************
	Fro

	From worker 29:	
	From worker 29:	******************************************************************************
	From worker 29:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 29:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 29:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 29:	******************************************************************************
	From worker 29:	
	From worker 3:	
	From worker 3:	******************************************************************************
	From worker 3:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 3:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 3:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 3:	******************************************************************************
	From work

In [ ]:
##############################################
## Moving Target

In [19]:
# Assume Ncell,Z exist, current ONLY WORK WITH THE LANDSCAPE IN PAPERS
# construct a 30x4x30x4 one step moving probability tensor

function build_c2c_tensor(matrix)
    

    
    # first, construct 120x120 one step transition matrix

    # here we number all cells from 1 to 120 in a row-major fashion
    c2c = zeros(Ncell * Z, Ncell * Z)
    for i = 1:(Ncell * Z)
        # Stay in current cell
        c2c[i,i] = matrix[2,2]

        # move right
        if mod(i,12) ==  0 # right end 
            c2c[i,i] = c2c[i,i] + matrix[2,3]
        else
            c2c[i,i+1] = matrix[2,3]
        end


        # move left
        if mod(i,12) ==  1 # left end 
            c2c[i,i] = c2c[i,i] + matrix[2,1]
        else
            c2c[i,i-1] = matrix[2,1]
        end

        # move down
        if i > 12 * (10-1) # bottom end
            c2c[i,i] = c2c[i,i] + matrix[3,2]
        else
            c2c[i,i+12] = matrix[3,2]
        end

        # move up
        if i <= 12 # top row
            c2c[i,i] = c2c[i,i] + matrix[1,2]
        else
            c2c[i,i-12] = matrix[1,2]
        end



        # move diagonal bottom-right
        if (mod(i,12) == 0) | (i > 12 * (10-1))
            c2c[i,i] = c2c[i,i] + matrix[3,3]
        else
            c2c[i,i+12+1] = matrix[3,3]
        end


        # move diagonal top-right
        if (mod(i,12) == 0) | (i <= 12)
            c2c[i,i] = c2c[i,i] + matrix[1,3]
        else
            c2c[i,i-12+1] = matrix[1,3]
        end

        # move diagonal top-left
        if (mod(i,12) == 1) | (i <= 12)
            c2c[i,i] = c2c[i,i] + matrix[1,1]
        else
            c2c[i,i-12-1] = matrix[1,1]
        end

        # move diagonal bottom-left
        if (mod(i,12) == 1) | (i > 12 * (10-1))
            c2c[i,i] = c2c[i,i] + matrix[3,1]
        else
            c2c[i,i+12-1] = matrix[3,1]
        end


    end

    # convert the 120x120 matrix into 30x4x30x4 tensor
    # Note that Julia is column major: suppose the first (leftmost, upmost) cell
    # is cell 1, cell 2 refers to the cell below cell 1 not the cell to the right
    # This affects how cell index is assigned
    c2c_tensor = zeros(Ncell,Z,Ncell,Z)
    for i=1:Ncell*Z
        zone_prev = (i / 60 > 1) * 2 + ((mod(i,12) / 6 > 1) | (mod(i,12) == 0)) + 1
        offset_prev = [0,6,60,66][zone_prev]
        cell_prev = (rem(i-offset_prev,12) - 1) * 5 + div(i-offset_prev,12) + 1 # column major in julia
        for j=1:Ncell*Z
            zone_cur = (j / 60 > 1) * 2 + ((mod(j,12) / 6 > 1) | (mod(j,12) == 0)) + 1
            offset_cur = [0,6,60,66][zone_cur]
            cell_cur = (rem(j-offset_cur,12) - 1) * 5 + div(j-offset_cur,12) + 1 # column major in julia
            c2c_tensor[cell_prev,zone_prev,cell_cur,zone_cur] = c2c[i,j]
        end
    end
    
    return c2c_tensor
end

build_c2c_tensor (generic function with 1 method)

In [20]:
# Myopic and Optimal Search Plan (without cross-cueing)

In [21]:
T = 4
Φ = 5
Total_Iter = 1 # the number of iterations to go from myopic search plan to optimal search plan

1

In [43]:


function mov_target_CE(prior,W,c2c_tensor)
    DTensor = ones(Ncell,Z,T)
    moving_target_obj_vec = zeros(Total_Iter,T)
    prob_nondetect_Tensor = zeros(Ncell,Z,T)
    final_assignment = zeros(S,Z,T)
    final_res_alloc = zeros(Ncell,Z,S,T)
    
    
    for i = 1:Total_Iter
        # Initialize UTensor
        UTensor = zeros(Ncell,Z,T)
        UTensor[:,:,1] =  prior


        for τ = 1:T
            if τ > 1
                UTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
            end
            β = UTensor[:,:,τ] .* DTensor[:,:,τ]
            obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,β,W)
            final_assignment[:,:,τ] = assignment
            final_res_alloc[:,:,:,τ] = res_alloc
            prob_nondetect_Tensor[:,:,τ] = prob_nondetect
            moving_target_obj_vec[i,τ] = obj
            print("Iter: ",i,". Completed timestep: ",τ,". Obj: ",obj,"\n")
        end

        # Compute the DTensors using the nondetection probabilities at each timestep as calculated
        # in the previous iteration
        for τ = T-1:-1:1
            DTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                    reshape(prob_nondetect_Tensor[:,:,τ+1],1,1,Ncell,Z) .* 
                    reshape(DTensor[:,:,τ+1],1,1,Ncell,Z),3),4)
        end
    end
    
    return moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc

end




function compute_non_detection(prior, W, c2c_tensor,final_assignment,final_res_alloc)
    # Compute nondetection probability at each time step (used in computing U and D)
    nondetection_prob_holder = zeros(Ncell,Z,T)
    for i=1:T
        res_alloc_cur = final_res_alloc[:,:,:,i]
        assign_cur = final_assignment[:,:,i]
        nondetection_prob_holder[:,:,i] = exp.( squeeze(-sum(W .* res_alloc_cur .* reshape(transpose(assign_cur),1,Z,S),3),3))
    end


    # Compute U and D

    UTensor_holder = zeros(Ncell,Z,T)
    UTensor_holder[:,:,1] =  prior
    for i=2:T

        UTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
            reshape(nondetection_prob_holder[:,:,i-1],Ncell,Z,1,1) .* 
            reshape(UTensor_holder[:,:,i-1],Ncell,Z,1,1),1),2)

    end

## For optimal solution, also need to incorporate DTensor, for myopic solution, do not incorporate DTensor
#     DTensor_holder = ones(Ncell,Z,T)
#     for i = T-1:-1:1
#         DTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
#                 reshape(nondetection_prob_holder[:,:,i+1],1,1,Ncell,Z) .* 
#                 reshape(DTensor_holder[:,:,i+1],1,1,Ncell,Z),3),4)
#     end
    
#     β_holder = UTensor_holder .* DTensor_holder
    
    β_holder = UTensor_holder
    return sum(sum(β_holder[:,:,:] .* nondetection_prob_holder[:,:,:],1),2)
end 
     
    
function compute_non_detection_test(prior, W, c2c_tensor,prob_nondetection_tensor,T)
    # Compute nondetection probability at each time step (used in computing U and D)



    # Compute U and D

    UTensor_holder = zeros(Ncell,Z,T)
    UTensor_holder[:,:,1] =  prior
    for i=2:T

        UTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
            reshape(prob_nondetection_tensor[:,:,i-1],Ncell,Z,1,1) .* 
            reshape(UTensor_holder[:,:,i-1],Ncell,Z,1,1),1),2)

    end


    β_holder = UTensor_holder
    return sum(sum(β_holder[:,:,:] .* prob_nondetection_tensor[:,:,:],1),2)
end 




compute_non_detection_test (generic function with 1 method)

In [58]:
# Modify the true moving distribution so I can meaningfully simulate longer time step with 
# target probably at the center of the landscape (10x12)
moving_mat= [0.0 0.0 0.0;
             0.0 0.5 0.125;
             0.0 0.125 0.25]
c2c_tensor_true = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")

moving_mat= ones(3,3)/9
c2c_tensor_random = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")


moving_mat= [0.25 0.125 0.0;
             0.125 0.5 0.0;
             0.0 0.0 0.0]
c2c_tensor_opposite = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")


# moving_mat= [0.0 0.125 0.25;
#              0.0 0.5 0.125;
#              0.0 0.0 0.0]
# c2c_tensor_opposite = build_c2c_tensor(moving_mat)
# print(sum(moving_mat),"\n")

1.0
1.0000000000000002
1.0


In [45]:
# When you have the true moving probability

T = 9
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
print(nondetection_no_movement_test,"\n")
toc()


Iter: 1. Completed timestep: 1. Obj: 0.7968924058025902
Iter: 1. Completed timestep: 2. Obj: 0.6369253334565347
Iter: 1. Completed timestep: 3. Obj: 0.5236403091695448
Iter: 1. Completed timestep: 4. Obj: 0.44932918665351196
Iter: 1. Completed timestep: 5. Obj: 0.39323606437509767
Iter: 1. Completed timestep: 6. Obj: 0.35054657779492415
Iter: 1. Completed timestep: 7. Obj: 0.3158754430550314
Iter: 1. Completed timestep: 8. Obj: 0.285086742313116
Iter: 1. Completed timestep: 9. Obj: 0.2559544074107196
[0.796892]

[0.636925]

[0.52364]

[0.449329]

[0.393236]

[0.350546]

[0.315875]

[0.285087]

[0.255954]
elapsed time: 35.987637416 seconds


35.987637416

In [56]:
# When you DO NOT have the true moving probability and we assume uniform moving probability

T = 9
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_random)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
print(nondetection_no_movement_test,"\n")
toc()


# Most of the time, actual ND > expected ND, but not always. because, we have a prior, then compute a allocation
# => ND tensor (minimize the expected ND). But given this ND tensor, starting prior might not be the
# expected ND minimizer associated with this ND tensor

Iter: 1. Completed timestep: 1. Obj: 0.8022573319295438
Iter: 1. Completed timestep: 2. Obj: 0.6878741542665952
Iter: 1. Completed timestep: 3. Obj: 0.609369441643152
Iter: 1. Completed timestep: 4. Obj: 0.5527555203974823
Iter: 1. Completed timestep: 5. Obj: 0.5098536546666245
Iter: 1. Completed timestep: 6. Obj: 0.4760043352258788
Iter: 1. Completed timestep: 7. Obj: 0.4489465248632808
Iter: 1. Completed timestep: 8. Obj: 0.4265350060485873
Iter: 1. Completed timestep: 9. Obj: 0.40732920648805626
[0.802257]

[0.665063]

[0.595735]

[0.524759]

[0.484972]

[0.472487]

[0.45354]

[0.443495]

[0.441603]
elapsed time: 31.110317383 seconds


31.110317383

In [60]:
# What if I assume a distribution that is the exact opposite of the true distribution

T = 9

tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_opposite)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
print(nondetection_no_movement_test,"\n")
toc()

# expected ND should not mirror those calculated using true moving distribution because the landscape in
# opposite direction is not mirrored

Iter: 1. Completed timestep: 1. Obj: 0.7968924057758602
Iter: 1. Completed timestep: 2. Obj: 0.6629948416765538
Iter: 1. Completed timestep: 3. Obj: 0.5731810521949596
Iter: 1. Completed timestep: 4. Obj: 0.5012389193706145
Iter: 1. Completed timestep: 5. Obj: 0.4453889879540731
Iter: 1. Completed timestep: 6. Obj: 0.4005655922095904
Iter: 1. Completed timestep: 7. Obj: 0.36205713694758673
Iter: 1. Completed timestep: 8. Obj: 0.326678222390394
Iter: 1. Completed timestep: 9. Obj: 0.2928990699719832
[0.796892]

[0.689151]

[0.624023]

[0.615138]

[0.612112]

[0.611122]

[0.610852]

[0.610822]

[0.610816]
elapsed time: 33.854640076 seconds


33.854640076

In [49]:
# What if I assume a distribution (which makes detection easier), but the true distribution is uniform (completely random)

T = 9
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_random,prob_nondetect_Tensor,T)
print(nondetection_no_movement_test,"\n")
toc()


Iter: 1. Completed timestep: 1. Obj: 0.7976995525482904
Iter: 1. Completed timestep: 2. Obj: 0.6322583022622119
Iter: 1. Completed timestep: 3. Obj: 0.5221293151593812
Iter: 1. Completed timestep: 4. Obj: 0.4474251520278681
Iter: 1. Completed timestep: 5. Obj: 0.39272373563248864
Iter: 1. Completed timestep: 6. Obj: 0.350316981314022
Iter: 1. Completed timestep: 7. Obj: 0.31551732255345144
Iter: 1. Completed timestep: 8. Obj: 0.28474288814654936
Iter: 1. Completed timestep: 9. Obj: 0.25576928044364083
[0.7977]

[0.684209]

[0.615023]

[0.576708]

[0.548538]

[0.530285]

[0.517801]

[0.509476]

[0.504462]
elapsed time: 30.785001074 seconds


30.785001074

In [ ]:
sum(unnorm)

In [29]:
# # Modify the true moving distribution so I can meaningfully simulate longer time step with 
# # target probably at the center of the landscape (10x12)
# moving_mat= [0.0 0.0 0.0;
#              0.0 0.5 0.125;
#              0.0 0.125 0.25]
# c2c_tensor_true = build_c2c_tensor(moving_mat)
# print(sum(moving_mat),"\n")



# moving_mat= [0.25 0.125 0.0;
#              0.125 0.5 0.0;
#              0.0 0.0 0.0]
# c2c_tensor_opposite = build_c2c_tensor(moving_mat)
# print(sum(moving_mat),"\n")

In [61]:
moving_mat= [0.0 0.125 0.25;
             0.0 0.5 0.125;
             0.0 0.0 0.0]
c2c_tensor_opposite = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")

1.0


In [94]:
UTensor = zeros(Ncell,Z,T)
UTensor[:,:,1] =  prior
        
UTensor_actual = zeros(Ncell,Z,T)
UTensor_actual[:,:,1] =  prior

prob_nondetect_Tensor = zeros(Ncell,Z,T)
actuals = []

0-element Array{Any,1}

In [95]:
τ = 1
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
print(sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ]),"\n")
push!(actuals,obj)
obj

0.7968924012499323


0.7968924057952709

In [96]:
τ = 2
UTensor[:,:,τ] =  sum(sum(c2c_tensor_opposite .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.6818703432532278
0.6698310287788398
1.0179736589634205

In [97]:
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

1×1×9 Array{Float64,3}:
[:, :, 1] =
 0.796892

[:, :, 2] =
 0.68187

[:, :, 3] =
 0.0

[:, :, 4] =
 0.0

[:, :, 5] =
 0.0

[:, :, 6] =
 0.0

[:, :, 7] =
 0.0

[:, :, 8] =
 0.0

[:, :, 9] =
 0.0

In [98]:
num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []

for i=1:num_CE_movementmet
    moving_mat = rand(Uniform(0,1), 3,3)
    moving_mat = moving_mat/sum(moving_mat)
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
    trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
end


In [99]:
summarystats(movement_obj.* 1.0)

Summary Stats:
Mean:           0.688957
Minimum:        0.643502
1st Quartile:   0.682012
Median:         0.688787
3rd Quartile:   0.696243
Maximum:        0.722120


In [101]:
Threshold_movement = 10

10

In [102]:
best_ind = sortperm(movement_obj)[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

3×3 Array{Float64,2}:
 0.0711157  0.155861   0.204813 
 0.0569365  0.177656   0.212328 
 0.0413195  0.0485561  0.0314142

In [ ]:
best_ind = sortperm(movement_obj)[num_CE_movementmet-Threshold_movement+1:num_CE_movementmet]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

In [ ]:
best_ind = sortperm(abs.(movement_obj - actual))[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

In [103]:
best_ind = sortperm((movement_obj - actual .<= 0) + (movement_obj - actual))[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

3×3 Array{Float64,2}:
 0.125037   0.137514   0.113852 
 0.0886206  0.13741    0.121492 
 0.115099   0.0943378  0.0666373

In [104]:

c2c_tensor_opposite = build_c2c_tensor(MV)
print(sum(moving_mat),"\n")

1.0000000000000002


In [105]:
movement_assign[:,:,best_ind]

3×3×10 Array{Float64,3}:
[:, :, 1] =
 0.0367555  0.00788189  0.181411
 0.0658555  0.181648    0.194336
 0.0987716  0.172424    0.060916

[:, :, 2] =
 0.0524761  0.119459  0.165555
 0.127515   0.113845  0.126961
 0.104955   0.085716  0.103519

[:, :, 3] =
 0.102737   0.165625  0.0806469
 0.0304964  0.140287  0.15259  
 0.154819   0.105901  0.0668966

[:, :, 4] =
 0.188766   0.237796   0.156629 
 0.0853273  0.026303   0.0368134
 0.113453   0.0229661  0.131946 

[:, :, 5] =
 0.180488   0.197111   0.00030606
 0.194253   0.172703   0.0919623 
 0.0907715  0.0210675  0.051339  

[:, :, 6] =
 0.191596   0.078806   0.102836 
 0.156038   0.0864887  0.192181 
 0.0921154  0.0881701  0.0117686

[:, :, 7] =
 0.180555    0.178347  0.041392  
 0.00474756  0.159394  0.139868  
 0.169219    0.116665  0.00981157

[:, :, 8] =
 0.0644515  0.0929632  0.16791  
 0.0405651  0.150798   0.145599 
 0.148367   0.142947   0.0463974

[:, :, 9] =
 0.135195   0.151909  0.0727366
 0.0332865  0.158345  0.131039 
 0.077

In [72]:
τ = 3
UTensor[:,:,τ] =  sum(sum(c2c_tensor_opposite .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.5819525994211606
0.5609759536500553
1.0373931282341398

In [73]:
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

1×1×9 Array{Float64,3}:
[:, :, 1] =
 0.7977

[:, :, 2] =
 0.673074

[:, :, 3] =
 0.581953

[:, :, 4] =
 0.0

[:, :, 5] =
 0.0

[:, :, 6] =
 0.0

[:, :, 7] =
 0.0

[:, :, 8] =
 0.0

[:, :, 9] =
 0.0

In [77]:
num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []

for i=1:num_CE_movementmet
    moving_mat = rand(Uniform(0,1), 3,3)
    moving_mat = moving_mat/sum(moving_mat)
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
end


In [78]:
best_ind = sortperm(movement_obj)[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

c2c_tensor_opposite = build_c2c_tensor(MV)
print(sum(moving_mat),"\n")
display(MV)

3×3 Array{Float64,2}:
 0.041928   0.110205   0.142654
 0.0483606  0.173311   0.189649
 0.0462812  0.0935984  0.154012

1.0


In [80]:
τ = 4
UTensor[:,:,τ] =  sum(sum(c2c_tensor_opposite .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.5309561847888082
0.49592780100844264
1.0706320228652986

In [81]:
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

1×1×9 Array{Float64,3}:
[:, :, 1] =
 0.7977

[:, :, 2] =
 0.673074

[:, :, 3] =
 0.581953

[:, :, 4] =
 0.530956

[:, :, 5] =
 0.0

[:, :, 6] =
 0.0

[:, :, 7] =
 0.0

[:, :, 8] =
 0.0

[:, :, 9] =
 0.0

In [82]:
num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []

for i=1:num_CE_movementmet
    moving_mat = rand(Uniform(0,1), 3,3)
    moving_mat = moving_mat/sum(moving_mat)
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
end


In [83]:
best_ind = sortperm(movement_obj)[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

c2c_tensor_opposite = build_c2c_tensor(MV)
print(sum(moving_mat),"\n")
display(MV)

3×3 Array{Float64,2}:
 0.06684    0.145632   0.142461
 0.0533047  0.17369    0.167703
 0.050015   0.0948321  0.105523

0.9999999999999998


In [84]:
τ = 5
UTensor[:,:,τ] =  sum(sum(c2c_tensor_opposite .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.4991340883967163
0.4501033153009284
1.108932263835931

In [85]:
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

1×1×9 Array{Float64,3}:
[:, :, 1] =
 0.7977

[:, :, 2] =
 0.673074

[:, :, 3] =
 0.581953

[:, :, 4] =
 0.530956

[:, :, 5] =
 0.499134

[:, :, 6] =
 0.0

[:, :, 7] =
 0.0

[:, :, 8] =
 0.0

[:, :, 9] =
 0.0

In [88]:
num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []

for i=1:num_CE_movementmet
    moving_mat = rand(Uniform(0,1), 3,3)
    moving_mat = moving_mat/sum(moving_mat)
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
end


In [89]:
best_ind = sortperm(movement_obj)[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

c2c_tensor_opposite = build_c2c_tensor(MV)
print(sum(moving_mat),"\n")
display(MV)

3×3 Array{Float64,2}:
 0.0988933  0.137692   0.177117
 0.0601485  0.105783   0.162905
 0.0441775  0.0886934  0.12459 

1.0000000000000002


In [90]:
τ = 6
UTensor[:,:,τ] =  sum(sum(c2c_tensor_opposite .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.4790297441382676
0.4159420481871932
1.1516742445877508

In [91]:
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

1×1×9 Array{Float64,3}:
[:, :, 1] =
 0.7977

[:, :, 2] =
 0.673074

[:, :, 3] =
 0.581953

[:, :, 4] =
 0.530956

[:, :, 5] =
 0.499134

[:, :, 6] =
 0.47903

[:, :, 7] =
 0.0

[:, :, 8] =
 0.0

[:, :, 9] =
 0.0

In [92]:
num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []

for i=1:num_CE_movementmet
    moving_mat = rand(Uniform(0,1), 3,3)
    moving_mat = moving_mat/sum(moving_mat)
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
end


In [93]:
best_ind = sortperm(movement_obj)[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

c2c_tensor_opposite = build_c2c_tensor(MV)
print(sum(moving_mat),"\n")
display(MV)

3×3 Array{Float64,2}:
 0.0964444  0.153248   0.17975  
 0.0543752  0.140829   0.151633 
 0.065972   0.0734731  0.0842759

1.0000000000000002


In [ ]:
τ = 7
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 8
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 9
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:

# When you have the true moving probability
T = 4
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_true = compute_non_detection(prior, W, c2c_tensor_true,final_assignment, final_res_alloc)
print(nondetection_true,"\n")
toc()




# When you DO NOT have the true moving probability and we assume uniform moving probability

T = 4
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_init)
nondetection_no_movement = compute_non_detection(prior, W, c2c_tensor_true,final_assignment, final_res_alloc)
print(nondetection_no_movement,"\n")
toc()


# What if I assume a distribution (which makes detection easier), but the true distribution is uniform
T = 4
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_no_movement_reverse = compute_non_detection(prior, W, c2c_tensor_init,final_assignment, final_res_alloc)
print(nondetection_no_movement_reverse,"\n")
toc()


In [ ]:
T = 10
moving_mat= ones(3,3)/9
c2c_tensor_init = build_c2c_tensor(moving_mat)

In [ ]:
UTensor = zeros(Ncell,Z,T)
UTensor[:,:,1] =  prior
        
UTensor_actual = zeros(Ncell,Z,T)
UTensor_actual[:,:,1] =  prior

prob_nondetect_Tensor = zeros(Ncell,Z,T)


In [ ]:
τ = 1
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
print(sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ]),"\n")
obj

In [ ]:
τ = 2
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 3
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 4
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 5
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 6
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 7
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 8
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
τ = 9
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

In [ ]:
# Currently only support myopic, total_iteration = 1.
# Eg. when I plan for T = 2, I use the non-detection at T = 1 based on actual movement distribution, 
# rather than the fictious non-detection based on my guessed movement distribution

function mov_target_CE_adaptive(prior,W,c2c_tensor_true,c2c_tensor_init)
    DTensor = ones(Ncell,Z,T)
    moving_target_obj_vec = zeros(Total_Iter,T)
    prob_nondetect_Tensor = zeros(Ncell,Z,T)
    final_assignment = zeros(S,Z,T)
    final_res_alloc = zeros(Ncell,Z,S,T)
    
    
    for i = 1:Total_Iter
        # Initialize UTensor
        UTensor = zeros(Ncell,Z,T)
        UTensor[:,:,1] =  prior
        
        UTensor_actual = zeros(Ncell,Z,T)
        UTensor_actual[:,:,1] =  prior


        for τ = 1:4
            if τ > 1
                UTensor[:,:,τ] = sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
                UTensor_actual[:,:,τ] = sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
            end
            β = UTensor[:,:,τ] .* DTensor[:,:,τ]
            obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,β,W)
            final_assignment[:,:,τ] = assignment
            final_res_alloc[:,:,:,τ] = res_alloc
            
            # Instead of using prob_nondetect, calculated with the guessed movement distribution, 
            # using actual outcome calculated with true movement distribution
            # 
            prob_nondetect_Tensor[:,:,τ] = prob_nondetect
#             prob_nondetect_Tensor[:,:,τ] = exp.(squeeze(-sum(W .* res_alloc .* reshape(assignment,1,Z,S),3),3))
            
            
            moving_target_obj_vec[i,τ] = obj
            print("Iter: ",i,". Completed timestep: ",τ,". Obj: ",obj,"\n")
        end

        # Compute the DTensors using the nondetection probabilities at each timestep as calculated
        # in the previous iteration
        for τ = T-1:-1:1
            DTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                    reshape(prob_nondetect_Tensor[:,:,τ+1],1,1,Ncell,Z) .* 
                    reshape(DTensor[:,:,τ+1],1,1,Ncell,Z),3),4)
        end
    end
    
    return moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc

end


In [ ]:
# ADAPTIVE: When you DO NOT have the true moving probability and we assume uniform moving probability
tic()

moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE_adaptive(prior,W,c2c_tensor_true,c2c_tensor_init)
nondetection_no_movement = compute_non_detection(prior, W, c2c_tensor_true,final_assignment, final_res_alloc)
print(nondetection_no_movement,"\n")


toc()

In [ ]:
# No prior Moving

num_trials = 1
T = 4
Total_Iter = 1
new_prior = ones(N,Z) .* 1/(N*Z)
nondetection_no_prior_moving = zeros(num_trials)
Φ = 5
tic()
for i=1:num_trials
    tic()
    moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(new_prior,W,c2c_tensor)
    nondetection_no_prior_moving[i] = compute_non_detection(prior, W, c2c_tensor,final_assignment, final_res_alloc)
    toc()
end
toc()
summarystats(nondetection_no_prior_moving)

In [ ]:
moving_mat_no= [1/9 1/9 1/9;
                 1/9 1/9 1/9;
                 1/9 1/9 1/9]
c2c_tensor_no = build_c2c_tensor(moving_mat_no)
sum(moving_mat_no)



In [ ]:
# moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_no)
# compute_non_detection(prior, W, c2c_tensor,final_assignment, final_res_alloc)



In [ ]:
# No moving assumption 

tic()
T = 4
Total_Iter = 2
num_trials = 50
nondetection_no_movement_moving = zeros(num_trials)
Φ = 5
for i=1:num_trials
    tic()
    moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_no)
    nondetection_no_movement_moving[i] = compute_non_detection(prior, W, c2c_tensor,final_assignment, final_res_alloc)
    toc()
end
toc()
summarystats(nondetection_no_movement_moving)